# Phase 14: Live Launch Preparation

**Goal**: Validate live paper trading readiness with dashboard, yield accrual, and softened thresholds.

## Key Features
- Real-time regime dashboard (fear/greed display)
- Auto-yield accrual with compounding (6.5% avg APY)
- Softened opportunistic short thresholds (RSI>68, ATR>4.2%, conf>0.78)
- Enhanced equity curve with yield metrics

## Changes from Phase 13
- RSI_OPPORTUNISTIC: 70 -> 68 (more opportunities)
- OPPORTUNISTIC_VOL_THRESHOLD: 4.5% -> 4.2% (lower bar)
- BEAR_CONFIDENCE_THRESHOLD: 0.75 -> 0.78 (quality filter)
- Added LiveDashboard class with fear/greed index
- Dashboard integration in live_paper.py

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from data_fetcher import DataFetcher
from portfolio import Portfolio
from orchestrator import RLOrchestrator
from dashboard import LiveDashboard

print("Phase 14: Live Launch Preparation")
print("=" * 50)

## 1. Dashboard Test

In [ ]:
# Test dashboard fear/greed calculation
dashboard = LiveDashboard(enable_plots=False)

# Test cases
test_cases = [
    (0.06, 75, 70, "High vol, high RSI = Fear"),
    (0.01, 25, 30, "Low vol, low RSI = Greed"),
    (0.03, 50, 50, "Normal vol, normal RSI = Neutral"),
    (0.05, 80, 75, "Bear regime = Extreme Fear"),
    (0.015, 20, 25, "Greed regime = Extreme Greed"),
]

print("Fear/Greed Index Tests:")
print("-" * 60)
for vol, rsi_xrp, rsi_btc, desc in test_cases:
    fg_index, fg_label = dashboard.get_fear_greed_index(vol, rsi_xrp, rsi_btc)
    print(f"{desc}")
    print(f"  Vol: {vol*100:.1f}%, RSI XRP: {rsi_xrp}, RSI BTC: {rsi_btc}")
    print(f"  -> Index: {fg_index:.1f}, Label: {fg_label}")
    print()

## 2. Fetch Current Market Data

In [ ]:
# Fetch latest data
fetcher = DataFetcher()
data = {}

for sym in ['XRP/USDT', 'BTC/USDT']:
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 500)
    if not df.empty:
        data[sym] = df
        print(f"{sym}: {len(df)} candles, latest close: ${df['close'].iloc[-1]:.4f}")

# Current prices
prices = {
    'USDT': 1.0,
    'XRP': data['XRP/USDT']['close'].iloc[-1],
    'BTC': data['BTC/USDT']['close'].iloc[-1]
}
print(f"\nCurrent Prices: XRP=${prices['XRP']:.4f}, BTC=${prices['BTC']:.2f}")

## 3. Validate Thresholds

In [ ]:
# Initialize portfolio and orchestrator
initial_balance = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}
portfolio = Portfolio(initial_balance.copy())

try:
    orchestrator = RLOrchestrator(portfolio, data)
    print("Orchestrator initialized successfully")
    print(f"RL Model loaded: {orchestrator.enabled}")
except Exception as e:
    print(f"Orchestrator init failed: {e}")
    orchestrator = None

if orchestrator:
    print("\nPhase 14 Thresholds:")
    print(f"  RSI_OPPORTUNISTIC: {orchestrator.RSI_OPPORTUNISTIC}")
    print(f"  OPPORTUNISTIC_VOL_THRESHOLD: {orchestrator.OPPORTUNISTIC_VOL_THRESHOLD*100:.1f}%")
    print(f"  BEAR_CONFIDENCE_THRESHOLD: {orchestrator.BEAR_CONFIDENCE_THRESHOLD}")
    print(f"  RSI_RIP_THRESHOLD: {orchestrator.RSI_RIP_THRESHOLD}")
    print(f"  BEAR_VOL_THRESHOLD: {orchestrator.BEAR_VOL_THRESHOLD*100:.1f}%")

## 4. Simulate Dec 7+ Live Trading

In [ ]:
# Simulate 24 hours of live trading (144 cycles at 10-min intervals)
if orchestrator and orchestrator.enabled:
    print("Simulating 24 hours of live trading...")
    print("=" * 60)
    
    sim_portfolio = Portfolio(initial_balance.copy())
    sim_orchestrator = RLOrchestrator(sim_portfolio, data)
    
    portfolio_values = []
    modes = []
    yields_earned = []
    
    # Use last 144 hourly candles (simulating 10-min at hourly data)
    n_steps = min(144, len(data['XRP/USDT']) - 50)
    
    for step in range(n_steps):
        # Update data window
        start_idx = step
        end_idx = start_idx + 50
        
        step_data = {
            'XRP/USDT': data['XRP/USDT'].iloc[start_idx:end_idx].reset_index(drop=True),
            'BTC/USDT': data['BTC/USDT'].iloc[start_idx:end_idx].reset_index(drop=True)
        }
        
        sim_orchestrator.data = step_data
        if sim_orchestrator.env:
            sim_orchestrator.env.data = step_data
        
        # Get current prices
        step_prices = {
            'USDT': 1.0,
            'XRP': step_data['XRP/USDT']['close'].iloc[-1],
            'BTC': step_data['BTC/USDT']['close'].iloc[-1]
        }
        
        # Run decision cycle
        result = sim_orchestrator.decide_and_execute(step_prices)
        sim_orchestrator.check_and_manage_positions(step_prices)
        sim_orchestrator.update_env_step()
        
        # Track results
        portfolio_value = sim_portfolio.get_total_usd(step_prices)
        portfolio_values.append(portfolio_value)
        modes.append(result.get('mode', 'unknown'))
        yields_earned.append(result.get('yield_earned', 0))
    
    # Results
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    total_yield = sum(yields_earned)
    pnl = final_value - initial_value
    roi = (pnl / initial_value) * 100
    
    print(f"\nSimulation Results ({n_steps} steps):")
    print(f"  Initial: ${initial_value:.2f}")
    print(f"  Final:   ${final_value:.2f}")
    print(f"  P&L:     ${pnl:+.2f} ({roi:+.2f}%)")
    print(f"  Yield:   ${total_yield:.4f}")
    print(f"\nMode Distribution:")
    for mode in set(modes):
        pct = modes.count(mode) / len(modes) * 100
        print(f"  {mode}: {pct:.1f}%")
else:
    print("Orchestrator not available for simulation")

## 5. Yield Projection

In [ ]:
# Project yield earnings at different USDT levels
apy = 0.065  # 6.5% avg

usdt_levels = [1000, 1500, 2000, 2500, 3000]
time_periods = {
    '1 day': 1/365,
    '1 week': 7/365,
    '1 month': 30/365,
    '1 year': 1
}

print("Yield Projections at 6.5% APY:")
print("=" * 70)
print(f"{'USDT':>10} | {'1 Day':>10} | {'1 Week':>10} | {'1 Month':>10} | {'1 Year':>10}")
print("-" * 70)

for usdt in usdt_levels:
    yields = [usdt * apy * frac for frac in time_periods.values()]
    print(f"${usdt:>9,.0f} | ${yields[0]:>9.2f} | ${yields[1]:>9.2f} | ${yields[2]:>9.2f} | ${yields[3]:>9.2f}")

## 6. Launch Checklist

In [ ]:
print("Phase 14 Live Launch Checklist:")
print("=" * 50)

checks = [
    ("RL Model loaded", orchestrator is not None and orchestrator.enabled if orchestrator else False),
    ("Data fetch working", len(data) == 2),
    ("Dashboard available", dashboard is not None),
    ("Thresholds softened (RSI>68)", orchestrator.RSI_OPPORTUNISTIC == 68 if orchestrator else False),
    ("Thresholds softened (ATR>4.2%)", orchestrator.OPPORTUNISTIC_VOL_THRESHOLD == 0.042 if orchestrator else False),
    ("YieldManager integrated", hasattr(portfolio, 'yield_manager')),
]

all_passed = True
for check, status in checks:
    symbol = "[PASS]" if status else "[FAIL]"
    print(f"  {symbol} {check}")
    if not status:
        all_passed = False

print()
if all_passed:
    print("All checks passed! Ready for live launch.")
    print("\nTo start live paper trading:")
    print("  python src/live_paper.py --interval 10")
else:
    print("Some checks failed. Please review before launching.")

## 7. Current Portfolio State

In [ ]:
# Calculate current portfolio value with real prices
starting_balance = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}

print("Starting Portfolio:")
print(f"  USDT: ${starting_balance['USDT']:.2f}")
print(f"  XRP:  {starting_balance['XRP']:.2f} @ ${prices['XRP']:.4f} = ${starting_balance['XRP'] * prices['XRP']:.2f}")
print(f"  BTC:  {starting_balance['BTC']:.6f} @ ${prices['BTC']:.2f} = ${starting_balance['BTC'] * prices['BTC']:.2f}")

total_value = (
    starting_balance['USDT'] +
    starting_balance['XRP'] * prices['XRP'] +
    starting_balance['BTC'] * prices['BTC']
)
print(f"\nTotal Portfolio Value: ${total_value:.2f}")
print(f"\nAllocation:")
print(f"  USDT: {starting_balance['USDT']/total_value*100:.1f}%")
print(f"  XRP:  {starting_balance['XRP']*prices['XRP']/total_value*100:.1f}%")
print(f"  BTC:  {starting_balance['BTC']*prices['BTC']/total_value*100:.1f}%")

## Summary

Phase 14 is ready for live paper trading deployment:

1. **Dashboard**: Fear/greed index calculated from volatility + RSI
2. **Thresholds**: Softened for Dec grind rips (RSI>68, ATR>4.2%, conf>0.78)
3. **Yield**: 6.5% avg APY auto-accrual on parked USDT
4. **Launch command**: `python src/live_paper.py --interval 10`

### Expected Behavior
- 100% defensive in high volatility (earns yield)
- Opportunistic shorts on RSI>68 + ATR>4.2% spikes
- Aggressive shorts on RSI>72 rips
- Target: Bears flat/positive via compounding yield + precision captures